In [ ]:
## Mini Dashboard map
import dash_daq as daq
import plotly
import plotly.express as px
import plotly.graph_objects as go
# import dash  # (version 1.12.0) pip install dash
#import dash_core_components as dcc
#import dash_html_components as html
from dash.dependencies import Input, Output

import numpy as np
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc

import pandas as pd
from dash import html
from dash import dcc

# app = dash.Dash(__name__ , external_stylesheets= [dbc.themes.CYBORG])
# app = JupyterDash(__name__, external_stylesheets= [dbc.themes.CYBORG])
app = JupyterDash(__name__, )


# Load and cleant the dataframe
df_res = pd.read_csv('sch_wmo_switch.csv', sep = '\t')
df_res = df_res.loc [ (df_res['delta_days'] != 'NA') &
                     (df_res['delta_days'] > 0) &
                     (df_res['sch'] != '-922') &
                     (df_res['wmo'] != 'NA ') &
                     (df_res['sch'] != 'NA ') &
                     (df_res['sch'] != df_res['wmo'] ) ]

df_res = df_res.reset_index()

# fill random vectors
# Must set compatible flag

df_res['compatible'] = 0

comp_df = pd.read_csv('compatible_wmo_sch.csv', sep='\t')

comp = []
for i in range(len(df_res)):
    s = df_res['sch'][i].replace(' ', '')
    w = df_res['wmo'][i].replace(' ', '')

    sc = comp_df.loc[ comp_df.sch == s ]
    if len(sc) >0:
        comp.append(1)
    else:
        comp.append(0)

df_res['compatible'] = comp

"""
import random
for i in [1,2,3]:
    ind = list(random.sample( list(range(len(df_res)) ) , int(len(df_res)/3)  ) )
    df_res.loc[ind, 'compatible'] = i
"""



app.layout = html.Div([
    # title of our web page
    html.H1("Sensors Ids - Map Dashboard"),
    html.Br(),  # Br is a break i.e. a space in between

    dcc.Tabs([

        dcc.Tab( label='Map' ,
             children=[
                    html.Div([
                        html.Div(children=[
                            daq.ToggleSwitch( id='toggle', value=False, size=100,
                                #vertical=True,
                                label = { 'label': 'Schroeder / WMO', 'style': {'font-size': 20} },
                                style={'width': '10%', 'marginLeft': 50,'marginBottom': 40, 'marginTop': 40,
                                       'display': 'inline-block'},
                            ),
                            dcc.RadioItems(
                                id = 'radio',
                                #options={ 'All': 'All Sensors', 'Compatible': 'Compatible', 'Incompatible': 'Incompatible' },
                                options={ 'Compatible': 'Compatible',
                                         'Incompatible': 'Incompatible'},

                                value = 'Compatible',
                                style = {'width': '49%', 'display': 'inline-block', 'marginLeft': 50,
                                      'padding-left': 40,  },
                                labelStyle={'display': 'block','text-align': 'justify', 'font-size': 25, },
                            ),
                            html.Br(), html.Br(),
                            html.Div([
                                    dcc.Graph(id='sensor_map', figure={},
                                              style={'display': 'inline-block', 'width': "70%",
                                                     'marginLeft': '-10px'}
                                              ),

                                    html.Div([ dcc.Graph(id='bar1', figure={},
                                              style={ 'width': "20%", 'marginLeft': '-10px'} ),
                                              dcc.Graph(id='bar2', figure={}, style={ 'width': "20%", 'marginLeft': '-10px'}
                                              )],
                                              style={'display': 'inline-block', 'width': "30%", 'marginLeft': '-10px'} )
                            ],
                                    style={'width': '100%', 'display': 'inline-block', },
                            ),
                        ]),
                    ])
            ]
        ),

        dcc.Tab( label='Info',
                 children=[
                     html.Div([
                         dcc.Graph(id='table', figure={},
                                   style={'display': 'inline-block',
                                          'width': "100%",
                                          'marginLeft': '-10px'}),
                     ],
                         style={'width': '100%',
                                'display': 'inline-block',
                                },
                         className="dash-container", )
                 ],
        ),
    ])

])

"""
@app.callback(
    [Output(component_id='sensor_map', component_property='figure'),
     Output(component_id='bar1',  component_property='figure' ),
     Output(component_id='bar2',  component_property='figure' ),
     Output(component_id='table', component_property='figure' )
     ],

    [Input('toggle', 'value'),
     Input('radio', 'value')]
)

def plots(toggle, radio):

    # mapping the compatible, incompatible, all stations
    comp_dic = {'All' : 2,
           "Compatible": 1,
           "Incompatible": 0 }

    df_res_ = df_res.loc[df_res['compatible'] == comp_dic[radio] ]

    df_res_.wmo = df_res_.wmo.astype(int).astype(str)
    # to select a color bar: https://plotly.com/python/discrete-color/

    if not toggle:
        id = "SCHROEDER"
        title = id + ' Sensor Ids'
        col = df_res_.sch
        cb = px.colors.qualitative.G10
    else:
        id = "WMO"
        title = id + ' Sensor Ids'
        col = df_res_.wmo
        cb = px.colors.qualitative.Plotly

    # Creating maps
    title = title + '  for ' + radio + '  Sensors'
    map = px.scatter_geo(df_res_,
                         lat=df_res_.latitude,
                         lon=df_res_.longitude,
                         hover_name="station",
                         hover_data={"sch" : True,
                                     "wmo" : True,
                                     'latitude': False,
                                     'longitude': False,
                                     'station_name': True},
                         color=col,
                         color_discrete_sequence= cb,
                         )

    # see bottom https://community.plotly.com/t/margins-around-graphs/11550/4
    map.update_layout(
        height=1000, width=1500,
        title= { 'text': title,  "yref": "paper","y": 1, "yanchor": "bottom" },
        #xaxis_title="X Axis Title",
        #yaxis_title="Y Axis Title",
        font=dict( family="Courier New, monospace", size=20, #color="RebeccaPurple"
        ),
        # margin=dict(l=20, r=20, t=0, b=5),
        legend=dict(font=dict(family="Courier New, monospace", size=25, color="black"), title = "Sensor Id"),
        legend_title=dict(font=dict(family="Courier New, monospace", size=25, color="blue"))
    )


    # Creating bar plots
    df_res_['wmo'] = df_res_.wmo.astype(str)

    bar_sch = px.histogram(df_res_, x="sch").update_xaxes(categoryorder='total descending')
    bar_sch.update_layout(
        height=500, width=500,
        title= { 'text': "Schroeder Id Counts",  "yref": "paper", "yanchor": "bottom" },
        font=dict( family="Courier New, monospace", size=20, #color="RebeccaPurple"
        ),
    )

    bar_wmo = px.histogram(df_res_, x="wmo").update_xaxes(categoryorder='total descending')
    bar_wmo.update_layout(
        height=500, width=500,
        title= { 'text': "WMO Id Counts",  "yref": "paper", "yanchor": "bottom" },
        font=dict( family="Courier New, monospace", size=20, #color="RebeccaPurple"
        ),
    )

    # Plotting the table
    df_res = df_res.sort_values(by=['delta_days'])
    table = go.Figure(data=[go.Table(
        header=dict(values=list(
            ['Station', 'Schroeder', 'Schroeder Date', 'WMO', 'WMO Date', 'Delta Days', 'Data Start', 'Data End',
             'Total Records', 'Compatible']),
                    fill_color='turquoise',
                    align='left',
                    font_size=16),
        columnwidth=[5, 3, 4, 3, 4, 3, 4, 4, 3],

        cells=dict(values=[df_res.station, df_res.sch, df_res.sch_date,
                           df_res.wmo, df_res.wmo_date,
                           df_res.delta_days,
                           df_res.min_date,
                           df_res.max_date,
                           df_res.records,
                           df_res.compatible,
                           ],

                   fill_color='aliceblue',
                   align='left',
                   font_size=15,
                   line_color='darkslategray',
                   fill=dict(color=['paleturquoise', 'white']),
                   height=25)

    )])

    table.update_layout(height=1000, width=1500)

    return [map, bar_sch, bar_wmo, table]

"""


@app.callback(
    [Output(component_id='sensor_map', component_property='figure'),
     Output(component_id='bar1',  component_property='figure' ),
     Output(component_id='bar2',  component_property='figure' ),
     ],

    [Input('toggle', 'value'),
     Input('radio', 'value')]
)

def plots(toggle, radio):

    # mapping the compatible, incompatible, all stations
    comp_dic = {'All' : 2,
           "Compatible": 1,
           "Incompatible": 0 }

    df_res_ = df_res.loc[df_res['compatible'] == comp_dic[radio] ]

    df_res_.wmo = df_res_.wmo.astype(int).astype(str)
    # to select a color bar: https://plotly.com/python/discrete-color/

    if not toggle:
        id = "SCHROEDER"
        title = id + ' Sensor Ids'
        col = df_res_.sch
        cb = px.colors.qualitative.G10
    else:
        id = "WMO"
        title = id + ' Sensor Ids'
        col = df_res_.wmo
        cb = px.colors.qualitative.Plotly

    # Creating maps
    title = title + '  for ' + radio + '  Sensors'
    map = px.scatter_geo(df_res_,
                         lat=df_res_.latitude,
                         lon=df_res_.longitude,
                         hover_name="station",
                         hover_data={"sch" : True,
                                     "wmo" : True,
                                     'latitude': False,
                                     'longitude': False,
                                     'station_name': True,
                                     'compatible':True,
                                     'delta_days':True},
                         color=col,
                         color_discrete_sequence= cb,
                         )

    # see bottom https://community.plotly.com/t/margins-around-graphs/11550/4
    map.update_layout(
        height=900, width=1200,
        title= { 'text': title,  "yref": "paper","y": 1, "yanchor": "bottom" },
        #xaxis_title="X Axis Title",
        #yaxis_title="Y Axis Title",
        font=dict( family="Courier New, monospace", size=20, #color="RebeccaPurple"
        ),
        # margin=dict(l=20, r=20, t=0, b=5),
        legend=dict(font=dict(family="Courier New, monospace", size=25, color="black"), title = "Sensor Id"),
        legend_title=dict(font=dict(family="Courier New, monospace", size=25, color="blue"))
    )


    # Creating bar plots
    df_res_['wmo'] = df_res_.wmo.astype(str)

    bar_sch = px.histogram(df_res_, x="sch").update_xaxes(categoryorder='total descending')
    bar_sch.update_layout(
        height=500, width=500,
        title= { 'text': "Schroeder Id Counts",  "yref": "paper", "yanchor": "bottom" },
        font=dict( family="Courier New, monospace", size=20, #color="RebeccaPurple"
        ),
    )

    bar_wmo = px.histogram(df_res_, x="wmo").update_xaxes(categoryorder='total descending')
    bar_wmo.update_layout(
        height=500, width=500,
        title= { 'text': "WMO Id Counts",  "yref": "paper", "yanchor": "bottom" },
        font=dict( family="Courier New, monospace", size=20, #color="RebeccaPurple"
        ),
    )
    return [map, bar_sch, bar_wmo]



@app.callback(
    [
     Output(component_id='table', component_property='figure' ),
     Input('toggle', 'value')
     ],

)

def make_table(toggle):

    df_ress = df_res.sort_values(by=['delta_days'])

    # replacing labels for table
    comp = [ 'True' if  v==1 else 'False' for v in df_ress.compatible.values ]
    df_ress['compatible'] = comp

    # Plotting the table

    table = go.Figure(data=[go.Table(
        header=dict(values=list(
            ['Station', 'Schroeder', 'Schroeder Date', 'WMO', 'WMO Date', 'Delta Days', 'Data Start', 'Data End',
             'Total Records', 'Compatible']),
                    fill_color='turquoise',
                    align='left',
                    font_size=16),
        columnwidth=[5, 3, 4, 3, 4, 3, 4, 4, 3, 3],

        cells=dict(values=[df_ress.station, df_ress.sch, df_ress.sch_date,
                           df_ress.wmo, df_ress.wmo_date,
                           df_ress.delta_days,
                           df_ress.min_date,
                           df_ress.max_date,
                           df_ress.records,
                           df_ress.compatible,
                           ],

                   fill_color='aliceblue',
                   align='left',
                   font_size=15,
                   line_color='darkslategray',
                   fill=dict(color=['paleturquoise', 'white']),
                   height=25)

    )])

    table.update_layout(height=1000, width=1500)

    return[table]

if __name__ == '__main__':
    #app.run_server(mode = 'inline', debug=True)
    import socket
    host = socket.gethostbyname(socket.gethostname())
    #app.run_server(port=8058, debug=True, host = host)
    app.run_server(debug=True, host = host)
    # app.run_server(host='0.0.0.0', debug=True)

